In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
print(train.shape)
train.head()

(22184, 2)


,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


In [3]:
sorted(os.listdir('train/')[:5])

['09a95c4.jpg', '2b48c51.jpg', '6d8ed1b.jpg', '8cf761f.jpg', 'e1e2bfe.jpg']

In [4]:
splitted = train['Image_Label'].str.split('_',expand = True)
train['label'] = splitted[1]
train['image_name'] = splitted[0]
train['mask'] = ~train['EncodedPixels'].isna()
train.head()

,Image_Label,EncodedPixels,label,image_name,mask
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...,Fish,0011165.jpg,True
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...,Flower,0011165.jpg,True
2,0011165.jpg_Gravel,NaN,Gravel,0011165.jpg,False
3,0011165.jpg_Sugar,NaN,Sugar,0011165.jpg,False
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...,Fish,002be4f.jpg,True


In [5]:
train.isna().sum()

Image_Label          0
EncodedPixels    10348
label                0
image_name           0
mask                 0
dtype: int64

In [6]:
train.image_name.nunique()

5546

In [7]:
mask_count_df = train.groupby('image_name').agg(np.sum).reset_index()
mask_count_df.sort_values('mask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

(5546, 2)


,image_name,mask
821,24dd99c.jpg,4.0
1885,562f80a.jpg,4.0
3260,944f10b.jpg,4.0
1872,55b539f.jpg,4.0
4464,cdf7242.jpg,4.0


In [8]:
train_ohe_df = train[~train['EncodedPixels'].isnull()]
classes = train_ohe_df['label'].unique()
train_ohe_df = train_ohe_df.groupby('image_name')['label'].agg(set).reset_index()

In [9]:
train_ohe_df.head()

,image_name,label
0,0011165.jpg,"{Flower, Fish}"
1,002be4f.jpg,"{Flower, Fish, Sugar}"
2,0031ae9.jpg,"{Flower, Fish, Sugar}"
3,0035239.jpg,"{Flower, Gravel}"
4,003994e.jpg,"{Fish, Sugar, Gravel}"


In [10]:
for class_name in classes:
    train_ohe_df[class_name] = train_ohe_df['label'].map(lambda x: 1 if class_name in x else 0)
print(train_ohe_df.shape)
train_ohe_df.head()

(5546, 6)


,image_name,label,Fish,Flower,Sugar,Gravel
0,0011165.jpg,"{Flower, Fish}",1,1,0,0
1,002be4f.jpg,"{Flower, Fish, Sugar}",1,1,1,0
2,0031ae9.jpg,"{Flower, Fish, Sugar}",1,1,1,0
3,0035239.jpg,"{Flower, Gravel}",0,1,0,1
4,003994e.jpg,"{Fish, Sugar, Gravel}",1,0,1,1


In [11]:
x_train_index,x_valid_index = train_test_split(mask_count_df.index,test_size = 0.2,random_state = 51,
                                               stratify = train_ohe_df['label'].map(lambda x: str(sorted(list(x)))))
print(x_train_index.shape)
print(x_valid_index.shape)

(4436,)
(1110,)


**Data Generator**

In [12]:
import keras,cv2
import albumentations as aug

Using TensorFlow backend.


In [13]:
def rle2mask(rle,input_shape):
    width,height = input_shape[:2]
    mask = np.zeros((width,height)).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]
    
    current_position = 0
    for index,start in enumerate(starts):
        mask[start:start+lengths[index]] == 1
        current_position += lengths[index]
        
    return mask.reshape(height,width).T

def mask(rles,input_shape,reshape = None):
    number_of_rles = len(rles)
    
    if reshape is not None:
        masks = np.zeros((*reshape,number_of_rles))
    else:
        masks = np.zeros((*input_shape,number_of_rles))
    
    for i,rle in enumerate(rles):
        if type(rle) is str:
            if reshape is not None:
                mask = rle2mask(rle,input_shape)
                reshaped_mask = cv2.resize(mask,reshape)
                masks[:,:,i] = reshaped_mask
            else:
                masks[:,:,i] = rle2mask(rle,input_shape)
    return masks

In [14]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self,idxs,df,target_df = None,mode = 'fit',
                base_path = 'train/',batch_size = 32,dim = (1400,2100),n_channels = 3,reshape = None,
                augment = False,n_classes = 4,random_state = 51,shuffle = True,graystyle = False):
        
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.idxs = idxs
        self.reshape = reshape
        self.n_channels = n_channels
        self.augment = augment
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.graystyle = graystyle
        
        self.on_epoch_end()
        np.random.seed(self.random_state)
    
    #number of batches per epoch
    def __len__(self):
        return int(np.floor(len(self.idxs)/self.batch_size))
    
    #generating one batch of data
    def __getitem__(self,index):
        
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        list_ids = [self.idxs[i] for i in indexes]
        
        X = self.__generate_X(list_ids)
        if self.mode == 'fit':
            y = self.__generate_y(list_ids)
            
            if self.augment:
                X,y = self.__augment_batch(X,y)
                
            return X,y
        elif self.mode =='predict':
            return X
        
        else:
            raise AttributeError('The mode parameter should be either "fit" or "predict"')
    
    #update indexes after every epoch
    
    def on_epoch_end(self):
        
        self.indexes = np.arange(len(self.idxs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self,list_ids):
        
        if self.reshape is None:
            X = np.empty((self.batch_size , *self.dim,self.n_channels))
        else:
            X = np.empty((self.batch_size,*self.reshape,self.n_channels))
        
        for i,id in enumerate(list_ids):
            im_name = self.df['image_name'].iloc[id]
            img_path = f'{self.base_path}/{im_name}'
            img = cv2.cvtColor(cv2.imread(img_path),cv2.COLOR_BGR2RGB)
            img = img.astype(np.float32) / 255.
            
            if self.reshape is not None:
                img = cv2.resize(img,self.reshape)
            
            X[i,] = img
        return X
    
    def __generate_y(self,list_ids):
        
        if self.reshape is None:
            y=  np.empty((self.batch_size,*self.dim,self.n_classes),dtype = int)
        else:
            y = np.empty((self.batch_size,*self.reshape,self.n_classes),dtype = int)
        
        for i,id in enumerate(list_ids):
            im_name = self.df['image_name'].iloc[id]
            image_df = self.target_df[self.target_df['image_name'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            
            if self.reshape is not None:
                masks = mask(rles,input_shape = self.dim,reshape = self.reshape)
        y[i,] = masks
        
        return y
    
    def __augment_batch(self,X,y):
        
        for i in range(X.shape[0]):
            X[i,],y[i,] = self.__random_transform(X[i,],y[i,])
        
        return X,y
    
    def __random_transform(self,X,y):
        
        transforms = aug.Compose([
            aug.HorizontalFlip(p=0.5),
            aug.VerticalFlip(p= 0.5),
            aug.GridDistortion(p=0.5)
        ])
        
        transformed = transforms(image = X,mask = y)
        augmented_image = transformed['image']
        augmented_label = transformed['mask']
        
        return augmented_image,augmented_label
        
        
    
    

In [15]:
train_gen = DataGenerator(x_train_index,df = mask_count_df,target_df=train,batch_size=16,
                         reshape=(256,256),
                         augment=True,
                         graystyle=False,
                         shuffle=True,
                         n_channels=3,
                         n_classes=train.label.nunique())

train_eval_gen = DataGenerator(x_train_index,df = mask_count_df,target_df=train,batch_size=16,
                         reshape=(256,256),
                         augment=False,
                         graystyle=False,
                         shuffle=True,
                         n_channels=3,
                         n_classes=train.label.nunique())

valid_gen = DataGenerator(x_valid_index,df = mask_count_df,target_df=train,batch_size=16,
                         reshape=(256,256),
                         augment=False,
                         graystyle=False,
                         shuffle=True,
                         n_channels=3,
                         n_classes=train.label.nunique())

In [16]:
print(len(train_gen),len(train_eval_gen),len(valid_gen))

277 277 69


**Precision_Recall AUC callback**

In [17]:
import multiprocessing
from keras.callbacks import Callback
from sklearn.metrics import precision_recall_curve,auc

In [18]:
def callback(Callback):
    def __init__(self,data_gen,num_workers = multiprocessing.cpu_count(),
            early_stopping_patience = 5,
            plateau_patience = 3,reductio_rate = 0.5,stage = 'train',checkpoints_path = 'checkpoints/'):
        super(Callback,self).__init__()
        self.data_generator = data_gen
        self.num_workers = num_workers
        self.class_names = ['Fish','Flower','Sugar','Gravel']
        self.history = [[] for _ in range(len(self.class_names) + 1)]
        self.early_stopping_patience = early_stopping_patience
        self.plateau_patience = plateau_patience
        self.reduction_rate = reductio_rate
        self.stage = stage
        self.best_pr_auc = -float('inf')
        if not os.path.exist(checkpoints_path):
            os.makedirs(checkpoints_path)
        self.checkpoints_path = checkpoints_path
        
    def compute_pr_curve(self,y_t,y_p):
        pr_auc_mean = 0
        print('\n'+'#'*30+'\n')
        for class_i in range(len(self.class_names)):
            precision,recall,_ = precision_recall_curve(y_t[:,class_i],y_p[:,class_i])
            pr_auc = auc(recall,precision)
            pr_auc_mean += pr_auc/len(self.class_names)
            print(f"PR AUC {self.class_names[class_i]}, {self.stage}: {pr_auc:.3f}\n")
            self.history[class_i].append(pr_auc)
        print('\n'+'#'*20+'\n' +'PR AUC mean, '+self.stage+':'+"%.3f" %pr_auc_mean+'\n'+'#'*20+'\n')
        
        self.history[-1].append(pr_auc_mean)
        return pr_auc_mean
    
    def is_patience_lost(self, patience):
        if len(self.history[-1]) > patience:
            best_performance = max(self.history[-1][-(patience + 1):-1])
            return best_performance == self.history[-1][-(patience + 1)] and best_performance >= self.history[-1][-1]  
    
    def early_stopping_check(self, pr_auc_mean):
        if self.is_patience_lost(self.early_stopping_patience):
            self.model.stop_training = True    
    
    def model_checkpoint(self, pr_auc_mean, epoch):
        if pr_auc_mean > self.best_pr_auc:
            # remove previous checkpoints to save space
            for checkpoint in glob.glob(os.path.join(self.checkpoints_path, 'classifier_epoch_*')):
                os.remove(checkpoint)
        self.best_pr_auc = pr_auc_mean
        self.model.save(os.path.join(self.checkpoints_path, f'classifier_epoch_{epoch}_val_pr_auc_{pr_auc_mean}.h5'))              
        print('\n'+'#'*20+'\n'+'Saved new checkpoint'+'\n'+'#'*20+'\n')
        
    def reduce_lr_on_plateau(self):
        if self.is_patience_lost(self.plateau_patience):
            new_lr = float(keras.backend.get_value(self.model.optimizer.lr)) * self.reduction_rate
            keras.backend.set_value(self.model.optimizer.lr, new_lr)
            print('\n'+'#'*20+'\n'+'Reduced learning rate to' +new_lr+'.'+'\n'+'#'*20+'\n')
            
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_generator(self.data_generator,
                                              verbose=1,
                                              workers=self.num_workers)
        y_true = self.data_generator.get_labels()
        # estimate AUC under precision recall curve for each class
        pr_auc_mean = self.compute_pr_auc(y_true, y_pred)
              
        if self.stage == 'val':
            # early stop after early_stopping_patience=4 epochs of no improvement in mean PR AUC
            self.early_stopping_check(pr_auc_mean)

            # save a model with the best PR AUC in validation
            self.model_checkpoint(pr_auc_mean, epoch)

            # reduce learning rate on PR AUC plateau
            self.reduce_lr_on_plateau()            
        
    def get_pr_auc_history(self):
        return self.history 

In [19]:
tr_callback = callback(train_eval_gen)
val_callback = callback(valid_gen)

**Architecture** **Training**

In [26]:
from keras.models import Model,Input
from keras.layers import Conv2D,MaxPooling2D,Conv2DTranspose,concatenate
import segmentation_models as seg_model
import keras.backend as K
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,CSVLogger

In [21]:
preprocess = seg_model.get_preprocessing('resnet50')
model = seg_model.Unet('resnet50',
                      input_shape=(256,256,3),
                      classes=4)

W1028 10:05:23.714293 140393299924800 deprecation_wrapper.py:119] From /home/paperspace/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1028 10:05:23.753597 140393299924800 deprecation_wrapper.py:119] From /home/paperspace/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1028 10:05:23.754646 140393299924800 deprecation_wrapper.py:119] From /home/paperspace/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1028 10:05:23.755392 140393299924800 deprecation_wrapper.py:119] From /home/paperspace/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v

In [22]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [23]:
model.compile('Adam',seg_model.losses.bce_dice_loss,[dice_coef])
# model.summary()

W1028 10:05:43.515039 140393299924800 deprecation_wrapper.py:119] From /home/paperspace/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1028 10:05:43.543133 140393299924800 deprecation.py:323] From /home/paperspace/.local/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [27]:
earlystopping = EarlyStopping(monitor='val_loss', 
                            mode='min', 
                             patience=5,
                             restore_best_weights=True,
                             verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              mode='min',
                              patience=3,
                              factor=0.5,
                              min_lr=1e-6,
                              verbose=1)
checkpoint = ModelCheckpoint('model.h5',save_best_only=True)
csv = CSVLogger('model.csv',append=False)
cb = [earlystopping,reduce_lr,checkpoint,csv]

In [28]:
model.fit_generator(train_gen,validation_data=valid_gen,callbacks=cb,epochs=1,verbose=1)

Epoch 1/1
277/277 [==============================] - 509s 2s/step - loss: -8473622424163808256.0000 - dice_coef: 1.5137 - val_loss: 4.2452 - val_dice_coef: 4.3012e-07


**Predictions**

In [43]:
from tqdm import tqdm_notebook

In [64]:
test_df = []
TEST_BATCH_SIZE = 500

In [49]:
test = os.listdir('test/')
print(len(test))

3698


In [50]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,Image_Label,EncodedPixels
0,002f507.jpg_Fish,1 1
1,002f507.jpg_Flower,1 1
2,002f507.jpg_Gravel,1 1
3,002f507.jpg_Sugar,1 1
4,0035ae9.jpg_Fish,1 1


In [51]:
splitted = submission['Image_Label'].str.split('_',expand = True)
submission['label'] = splitted[1]
submission['image_name'] = splitted[0]
submission.head()

,Image_Label,EncodedPixels,label,image_name
0,002f507.jpg_Fish,1 1,Fish,002f507.jpg
1,002f507.jpg_Flower,1 1,Flower,002f507.jpg
2,002f507.jpg_Gravel,1 1,Gravel,002f507.jpg
3,002f507.jpg_Sugar,1 1,Sugar,002f507.jpg
4,0035ae9.jpg_Fish,1 1,Fish,0035ae9.jpg


In [52]:
submission.shape

(14792, 4)

In [54]:
test_images = pd.DataFrame(submission['image_name'].unique(),columns = ['image_name'])
test_images.head()

,image_name
0,002f507.jpg
1,0035ae9.jpg
2,0038327.jpg
3,004f759.jpg
4,005ba08.jpg


In [55]:
test_images.shape

(3698, 1)

In [57]:
best_threshold = 0.5
best_size = 25000

In [58]:
def mask2rle(img):
    pixels= img.T.flatten() 
    pixels = np.concatenate([[0], pixels, [0]])  # add 0 to the beginning and end of array
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rles(masks,reshape = None):
    width, height, depth = masks.shape
    
    rles = []
    
    for i in range(depth):
        mask = masks[:, :, i]
        
        if reshape:
            mask = mask.astype(np.float32)
            mask = cv2.resize(mask, reshape).astype(np.int64)
        
        rle = mask2rle(mask)
        rles.append(rle)
        
    return rles

In [59]:
def post_process(probability, threshold, min_size):
    
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((1400, 2100), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [60]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [65]:
encoded_pixels = []
for i in tqdm_notebook(range(0,len(test_images),TEST_BATCH_SIZE)):
    batch_idx = list(range(i,min(len(test_images),i+TEST_BATCH_SIZE)))
    test_gen = DataGenerator(batch_idx,df = test_images,shuffle=False,mode='predict',
                            reshape=(256,256),
                            n_channels=3,
                            graystyle=False,
                            base_path='test/',
                            target_df=submission,
                            batch_size=1,
                            n_classes=4)
    batch_pred_masks = model.predict_generator(test_gen,workers=1,verbose=1)
    
    for j, idx in enumerate(batch_idx):
        filename = test_images['image_name'].iloc[idx]
        image_df = submission[submission['image_name'] == filename].copy()
        
        # Batch prediction result set
        pred_masks = batch_pred_masks[j, ].round().astype(int)
        pred_rles = rles(pred_masks, reshape=(1400, 2100))
        
        image_df['EncodedPixels'] = pred_rles
        
        test_df.append(image_df)
        
        
        for k in range(pred_masks.shape[-1]):
            pred_mask = pred_masks[...,k].astype('float32') 
            
            if pred_mask.shape != (1400, 2100):
                pred_mask = cv2.resize(pred_mask, dsize=(2100, 1400), interpolation=cv2.INTER_LINEAR)
                
            pred_mask, num_predict = post_process(sigmoid(pred_mask), best_threshold, best_size )
            
            if num_predict == 0:
                encoded_pixels.append('')
            else:
                r = mask2rle(pred_mask)
                encoded_pixels.append(r)
    

198/198 [==============================] - 12s 62ms/step



In [66]:
len(encoded_pixels),len(submission)

(14792, 14792)

In [67]:
submission['EncodedPixels'] = encoded_pixels

In [69]:
submission.drop(['label','image_name'],axis = 1,inplace=True)

In [70]:
submission.head()

,Image_Label,EncodedPixels
0,002f507.jpg_Fish,1 2940000
1,002f507.jpg_Flower,1 1615493 1615544 1350 1616944 1350 1618344 13...
2,002f507.jpg_Gravel,1 2940000
3,002f507.jpg_Sugar,1 1615521 1615527 1395 1616927 1395 1618327 13...
4,0035ae9.jpg_Fish,1 2940000


In [71]:
submission.to_csv('unet_resnet_50_1_epoch.csv',index = False)

In [72]:
!kaggle competitions submit -c understanding_cloud_organization -f unet_resnet_50_1_epoch.csv -m "unet_resnet_50_1_epoch"

100%|████████████████████████████████████████| 174M/174M [00:29<00:00, 6.10MB/s]
Successfully submitted to Understanding Clouds from Satellite Images